# Importing stuffs

In [1]:
import numpy as np
import argparse
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array, array_to_img
from keras.utils import to_categorical
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.utils import resample
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf

# Set device placement to CPU
tf.config.set_visible_devices([], 'GPU')

# Rest of your code for model training


## Setting the mode to either train or display and changing the batch size and number of epochs

In [2]:
mode = "train"
batch_size = 32
num_epoch = 50

## Transforming the images for training into lists

In [3]:
main_folder = '/Users/nichdylan/Documents/Image Processing/images/train'

image_lists = {}

for subfolder in os.listdir(main_folder):
    subfolder_path = os.path.join(main_folder, subfolder)
    
    if os.path.isdir(subfolder_path) and subfolder != ".DS_Store":
        image_list = []
        
        for filename in os.listdir(subfolder_path):
            if filename.endswith('.jpg') or filename.endswith('.png'):
                image_path = os.path.join(subfolder_path, filename)
                image = cv2.imread(image_path)
                
                if image is not None:
                    image_list.append(image)

        image_lists[subfolder] = image_list

## Oversampling the disgust class

#### Number of "disgust" images before oversampling

In [4]:
print(len(image_lists['disgust']))

436


In [5]:
image_lists['disgust'] = resample(image_lists['disgust'], n_samples=len(image_lists['fear']), random_state=42)

#### Number of "disgust" images after oversampling

In [6]:
print(len(image_lists['disgust']))

4103


## Checking the number of images for each of the 7 classes

In [7]:
for list_name, list_data in image_lists.items():
    data_type = type(list_data)
    list_length = len(list_data)
    print(f"List Name: {list_name}, Type: {data_type}, Length: {list_length}")
    

List Name: happy, Type: <class 'list'>, Length: 7164
List Name: sad, Type: <class 'list'>, Length: 4938
List Name: fear, Type: <class 'list'>, Length: 4103
List Name: surprise, Type: <class 'list'>, Length: 3205
List Name: neutral, Type: <class 'list'>, Length: 4982
List Name: angry, Type: <class 'list'>, Length: 3993
List Name: disgust, Type: <class 'list'>, Length: 4103


## Preprocess the image (grayscale, rescale, change label formatting)

In [9]:
data = []
labels = []

for emotion, image_array_list in image_lists.items():
    data.extend(image_array_list)
    labels.extend([emotion] * len(image_array_list))

data = np.array(data)
labels = np.array(labels)

target_size = (48, 48)
resized_data = [array_to_img(img, data_format="channels_last").resize(target_size) for img in data]

resized_data = [img_to_array(img, data_format="channels_last") for img in resized_data]
resized_data = np.array(resized_data)

# A colored list would have a shape of (32488, 48, 48, 3)
# Make a list for grayscaled_images, it would have the shape of (32488, 48, 48, 1)
grayscale_images = []

# Loop through the colored images and convert them to grayscaled
for color_image in resized_data:
    grayscale_image = cv2.cvtColor(color_image, cv2.COLOR_RGB2GRAY)
    grayscale_image = np.expand_dims(grayscale_image, axis=-1)
    grayscale_images.append(grayscale_image)

# Convert the list of grayscale images to a NumPy array
grayscale_images = np.array(grayscale_images)


emotion_dict = {
    "angry": 0,
    "disgust": 1,
    "fear": 2,
    "happy": 3,
    "neutral": 4,
    "sad": 5,
    "surprise": 6
}

# Map emotion names to numerical values
numeric_labels = np.array([emotion_dict[label] for label in labels])
numeric_labels = to_categorical(numeric_labels, num_classes=7)  # Assuming you have 7 classes

Grayscale images shape: (32488, 48, 48, 1)


DONEE
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]
X shape: (32488, 48, 48, 1)
y shape: (32488, 7)


## Create the training generator

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    zoom_range=0.1
)

train_generator = train_datagen.flow(
    x=grayscale_images,  # Image data
    y=numeric_labels,  # Labels (emotions)
    batch_size=batch_size
)

## Create the validation generator

In [10]:
val_dir = 'images/validation'

num_train = 32533
num_val = 7178

val_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=(48,48),
        batch_size=batch_size,
        color_mode="grayscale",
        class_mode='categorical')


Found 7066 images belonging to 7 classes.


## Creating the model

In [12]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))

## When mode = train, it will train the model. If model = display, it will open up the camera and recognize the emotions

In [13]:
if mode == "train":
    model.compile(loss='categorical_crossentropy',optimizer=Adam(learning_rate=0.0001),metrics=['accuracy'])
    model_info = model.fit(
            train_generator,
            steps_per_epoch=num_train // batch_size,
            epochs=num_epoch,
            validation_data=validation_generator,
            validation_steps=num_val // batch_size)
    model.save('rotate_num_train_changed_32_batch_OS_exported_model_git.h5')
    
# emotions will be displayed on your face from the webcam feed
elif mode == "display":
    model.load_weights('exported_model_git.h5')

    # prevents openCL usage and unnecessary logging messages
    cv2.ocl.setUseOpenCL(False)

    # dictionary which assigns each label an emotion (alphabetical order)
    emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}

    # start the webcam feed
    cap = cv2.VideoCapture(0)
    while True:
        # Find haar cascade to draw bounding box around face
        ret, frame = cap.read()
        if not ret:
            break
        facecasc = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = facecasc.detectMultiScale(gray,scaleFactor=1.3, minNeighbors=5)

        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (255, 0, 0), 2)
            roi_gray = gray[y:y + h, x:x + w]
            cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray, (48, 48)), -1), 0)
            prediction = model.predict(cropped_img)
            maxindex = int(np.argmax(prediction))
            cv2.putText(frame, emotion_dict[maxindex], (x+20, y-60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        cv2.imshow('Video', cv2.resize(frame,(1600,960),interpolation = cv2.INTER_CUBIC))
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

Epoch 1/50
1016/1016 [==============================] - ETA: 0s - loss: 1.8734 - accuracy: 0.2472WARNING:tensorflow:Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 224 batches). You may need to use the repeat() function when building your dataset.


1016/1016 [==============================] - 95s 94ms/step - loss: 1.8734 - accuracy: 0.2472 - val_loss: 1.7585 - val_accuracy: 0.3108
Epoch 2/50
1016/1016 [==============================] - 102s 100ms/step - loss: 1.7002 - accuracy: 0.3485
Epoch 3/50
1016/1016 [==============================] - 117s 115ms/step - loss: 1.5774 - accuracy: 0.4036
Epoch 4/50
1016/1016 [==============================] - 120s 118ms/step - loss: 1.4861 - accuracy: 0.4424
Epoch 5/50
1016/1016 [==============================] - 133s 131ms/step - loss: 1.4088 - accuracy: 0.4742
Epoch 6/50
1016/1016 [==============================] - 132s 130ms/step - loss: 1.3514 - accuracy: 0.4951
Epoch 7/50
1016/1016 [==============================] - 151s 149ms/step - loss: 1.3002 - accuracy: 0.5165
Epoch 8/50
1016/1016 [==============================] - 128s 126ms/step - loss: 1.2607 - accuracy: 0.5324
Epoch 9/50
1016/1016 [==============================] - 138s 135ms/step - loss: 1.2259 - accuracy: 0.5446
Epoch 10/50
1016/